##### import libs

In [1]:
import numpy as np
import pandas as pd
import implicit
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split

In [2]:
from utils import evaluate, load_data

##### read data

In [3]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

##### split data

In [4]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [5]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [6]:
sparse_train = coo_matrix((likes_train-dislikes_train, (u_train, i_train)))
sparse_val = coo_matrix((likes_val-dislikes_val, (u_val, i_val)))

##### als model

In [ ]:
model = implicit.nearest_neighbours.ItemItemRecommender(,random_state=42)
model.fit(sparse_train)

##### evaluation

In [ ]:
model = model.to_cpu()

In [15]:
als_score = (model.user_factors[ui_val['user_id']] * 
             model.item_factors[ui_val['item_id']]).sum(axis=1)

In [16]:
np.save("dumps/bpr_val_score.npy", als_score)

In [17]:
ui_val['als_score'] = als_score

In [18]:
ui_val["target"] = ui_val["like"].astype(np.int8) - ui_val["dislike"].astype(np.int8)

In [19]:
evaluate(user_id=ui_val.user_id.values,
         target=ui_val.target.values,
         score=ui_val.als_score.values)

100%|██████████| 181721/181721 [00:51<00:00, 3508.06it/s]


0.5272686150733121

##### submission

In [ ]:
test_als_score = (model.user_factors[test_pairs_data['user_id']] * 
                  model.item_factors[test_pairs_data['item_id']]).sum(axis=1)

In [ ]:
test_pairs_data["predict"] = test_als_score
test_pairs_data.to_csv("./submission.csv",index=False)